# Langchain Tutorial
[Github](https://github.com/langchain-ai/langchain)  
[Youtube](https://www.youtube.com/watch?v=LBNpyjcbv0o&list=PLMxvXkOE4ezBznUjr3bYUgzdE-ZqIzMcM&index=1&t=1856s)

## Environment
```
conda create --name langchain python=3.8
conda activate langchain
pip install langchain
pip install langchain-openai
pip install beautifulsoup4
pip install faiss-cpu
```

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "xxxxxx"

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [ ]:
llm.invoke("what is docker and how is it useful for deployment?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-Korean translator that return whatever the user says in Korean."),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({"input" : "I enjoy going to rock concerts."})

In [ ]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input" : "My friend Robert has a blue cat"})

In [ ]:
llm.invoke("what is new in langchain 0.1.0?")

### Retrieval chain

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langchain-v0-1-0/")

docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
documents

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)

### Create chain for documents

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What is langchain 0.1.0?",
    "context": [Document(page_content="langchain 0.1.0 is the new version of a llm app development framework.")]
})

### Create retrieval chain

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "What is new in langchain 0.1.0?"
})

In [ ]:
response['answer']

### Conversational retrieval chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversations, generate a search query to look up in order to get information relevant to the conversations.")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it"
})

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", {input})
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke(
    "chat_history": [],
    "input": "What is langchain 0.1.0 about?"
)

In [ ]:
response['answer']

In [ ]:
response = conversational_retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it"
})

In [ ]:
response['answer']